In [17]:
import os
import pandas as pd
from collections import Counter
import plotly.graph_objects as go
import numpy as np
from datetime import date

# Подготовка

In [18]:
file = (os.path.abspath('BPLA, series 1 - инт.csv')).replace('\\', '/')
df = pd.read_csv(file, sep = ',')
df = pd.DataFrame(df.iloc[:,1:])

In [19]:
mx = np.array(df.loc[:, 'mx, Н*м'].values)
fx = np.array(df.loc[:, 'Fx, Н'].values)
w = np.array(df.loc[:, 'Wв, Вт'].values)
p = np.array(df.loc[:, 'P, Н/Вт'].values)

### Определим размер матрицы для графика

In [20]:
                                    # Используется при не одинаковом числе расчётов
for number, n_number in Counter(df['a, град']).items():
    break
for number, a_number in Counter(df['n, об/мин']).items():
    break
print(a_number, n_number)

39 11


### Подготовим матрицы для графиков 

In [21]:
k = 0

mx_m, fx_m, w_m, p_m = np.zeros((a_number, n_number)), np.zeros((a_number, n_number)), np.zeros((a_number, n_number)), np.zeros((a_number, n_number))
for i in range (0, a_number):
    for j in range (0, n_number):
        mx_m[i,j], fx_m[i,j], w_m[i,j], p_m[i,j] = mx[k], fx[k], w[k], p[k]
        k += 1

# Построение графиков

In [22]:
def plot(z_An_matrix, z_An_string, title_left, color):
    fig = go.Figure()
    fig = go.Figure()
    opa_sur = 0.95
    opa_mesh = 0.4
    opa_0_mesh = 0.3
    opa_0_plane = 0.3

    x = list(range(int(df['n, об/мин'].min()), int(df['n, об/мин'].max())+1000,1000))
    y = list(range(0, a_number+1, 1))

    fig.add_trace(go.Surface(x=x, y=y, z = z_An_matrix, 
                    opacity=opa_sur, showscale=False, showlegend=False,
                    name = 'a',
                    # colorbar_title = title,
                    legendgroup="group", 
                    colorscale=color
                                ))

    #                                                             Сетка
    i = 0
    j = 0
    k = 0
    z1 = np.zeros((n_number))
    for i in range (0, a_number):
        for j in range(0,n_number):
            z1[j] = z_An_string[k]  
            k += 1
        fig.add_trace(go.Scatter3d(x=list(range(0,int(df['n, об/мин'].max())+1000 ,1000)), y=[i]*n_number, z=z1, 
                                showlegend=False, legendgroup="group", opacity = opa_mesh,
                               marker=dict(size=1,color='black',colorscale='Viridis',),
                               line=dict(color='black', width=3)))
        fig.add_trace(go.Scatter3d(x=[int(df['n, об/мин'].max()) - i*1000]*a_number, y=list(range(0,a_number)), 
                               z=np.array(df.loc[(df.loc[:, 'n, об/мин']==int(df['n, об/мин'].max())-i*1000), 
                                                title_left]), showlegend=False, legendgroup="group", opacity = opa_mesh,
                                                marker=dict(size=1,color='black',colorscale='Viridis',),
                                                line=dict(color='black', width=3)))     

#_________________________________________________________________________________________________________________________   

                                              #Оформление 

    #Для первой сцены
    tickf = 12
    fig.update_layout(autosize=False,
                  scene = {"aspectratio": {"x": 1, "y": 1, "z": 1},  #Масштаб осей
                  'camera_eye': {"x": -2, "y": 2, "z":1.65},
                'camera_center' : {"x": 0, "y": 0, "z":0},}, 
                  width=950, height=500, #Размеры окна
                  margin=dict(l=10, r=0, b=10, t=50),                  
    )

    fig.update_layout(scene=dict(xaxis=dict(title="n, об/мин",
                        backgroundcolor="rgb(200, 200, 230)",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white", tickfont=dict(size=tickf)),
    yaxis=dict(title="a, град", 
                        backgroundcolor="rgb(230, 200,230)",
                        gridcolor="white",
                        showbackground=True,tickfont=dict(size=tickf),
                        zerolinecolor="white"),
    zaxis=dict(title=title_left, backgroundcolor="rgb(200, 200,200)",
                         gridcolor="white",
                         showbackground=True,tickfont=dict(size=tickf),
                         zerolinecolor="white",)),
                    margin=dict(r=20, b=10, l=0, t=30))
    fig.update_layout(scene=dict(xaxis_showspikes=False, yaxis_showspikes=False),) #Изменение при наведении мышки
    fig.update_scenes(camera_projection_type="orthographic") #вводит ортогональный вид

    
 #__________________________________________________________    
                            #Легенда

    fig.update_layout(showlegend=True)
    fig.update_layout(legend=dict(    #Изменение положения легенды
    yanchor="top",
    y=1,
    xanchor="left",
    x=0.1, 
    title_font_family="Times New Roman",
#     bgcolor=None
    ))   

    fig.update_layout(autosize=False,width=700,height=600,
        margin=dict(l=50, r=50,b=50,t=50,pad=4))
        
    fig.update_layout(
        title=f"Лопасть 20 х 4 х 150 мм;  {date.today()}",
        legend_title="Legend Title",
        )

    fig.update_layout(           #Позиционирование заголовка
        title={
           'text': f"Лопасть 20 х 4 х 150 мм; Vн = 0 м/с; {date.today()}",
           'y':0.97,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'})

    fig.show()                                

In [23]:
plot(fx_m, fx, 'Fx, Н', [[0,'rgb(255, 0, 0)'], [1,  'rgb(255, 0, 0)']])

In [24]:
plot(mx_m, mx, 'mx, Н*м', [[0,'rgb(255, 255, 0)'], [1,  'rgb(255, 255, 0)']])

In [25]:
plot(w_m, w, 'Wв, Вт', [[0,'rgb(66, 170, 255)'], [1,  'rgb(66, 170, 255)']])

In [26]:
plot(p_m, p, 'P, Н/Вт', [[0,'rgb(0, 0, 255)'], [1,  'rgb(0, 0, 255)']])